# 04 Surface normal

* 구 표면의 법선 벡터(normal vector)를 계산하고 색상으로 변환해 렌더링 합니다.

In [1]:
import numpy as np
import numpy.linalg as LA

import matplotlib.pyplot as plt

In [2]:
def normalize(vec: np.ndarray, eps: float = 1e-6):
    return vec / (LA.norm(vec) + eps)


def blend(color1: np.ndarray, color2: np.ndarray, t: float) -> np.ndarray:
    return (1.0 - t) * color1 + t * color2

In [3]:
class Ray:
    def __init__(self, origin: np.ndarray, direction: np.ndarray) -> None:
        self.origin: np.ndarray = origin
        self.direction: np.ndarray = normalize(direction)

    def at(self, t: float) -> np.ndarray:
        return self.origin + t * self.direction

* 구 표면의 법선 벡터 계산
  * Ray가 구와 만나는 경우 $t$를 계산합니다.
    * 카메라의 시작점에서 가까운 점을 계산하가 위해 아래의 근을 사용합니다.
$$t = \frac{-h - \sqrt{h^2 - ac}}{a}$$
  * 구의 중심 $\mathbf{C}$에서 구 표면의 점 $\mathbf{P}(t)$으로 향하는 벡터를 계산합니다.

<center>
<img src="./images/surface_normal.png" width="500px" height="500px" title="broadcasting"/>
</center>

In [4]:
def hit_sphere(center: np.ndarray, radius: float, ray: Ray) -> float:
    dir_center_to_origin = ray.origin - center

    a = np.dot(ray.direction, ray.direction)
    half_b = np.dot(dir_center_to_origin, ray.direction)
    c = np.dot(dir_center_to_origin, dir_center_to_origin) - radius**2.0

    discriminant = half_b**2 - a*c
    if discriminant < 0:
        return -1.0
    else:
        return (-half_b - np.sqrt(discriminant))/a

* 계산된 법선 벡터는 정규화(normalize)합니다.
* 법선 벡터의 각 $x, y, z$의 값의 범위 -1 ~ 1 을 0 ~ 1로 변환합니다. -1 ~ 1 범위의 값을 RGB 색상으로 표현하기 위해 많이 사용되는 방법입니다.
    * 0 이하의 컬러 값은 시각화 툴에 따라 0으로 클립핑 되기 때문에 차이를 제대로 확인하기 위해 0 ~ 1 로 변환합니다.

In [5]:
def ray_color(ray: Ray) -> np.ndarray:
    sphere_center = np.array([0, 0, -1])
    t = hit_sphere(center=sphere_center, radius=0.5, ray=ray)

    if t >= 0.0:
        N = normalize(ray.at(t) - sphere_center)
        return 0.5 * (N + 1.0)

    t = 0.5 * (ray.direction[1] + 1.0)
    color1 = np.array([1.0, 1.0, 1.0])
    color2 = np.array([0.5, 0.7, 1.0])
    return blend(color1=color1, color2=color2, t=t)

이후의 구현은 이전과 동일합니다.

In [6]:
# image
aspect_ratio = 16.0 / 9.0
image_width = 320
image_height = int(image_width / aspect_ratio)

image = np.zeros([image_height, image_width, 3])

In [7]:
# camera
viewport_height = 2.0
viewport_width = aspect_ratio * viewport_height
focal_length = 1.0

camera_origin = np.array([0.0, 0.0, 0.0])
horizontal_vec = np.array([viewport_width, 0.0, 0.0])
vertical_vec = np.array([0.0, viewport_height, 0.0])
frontal_vec = np.array([0.0, 0.0, focal_length])
lower_left_corner = (
    camera_origin - horizontal_vec / 2.0 - vertical_vec / 2.0 - frontal_vec
)

pixel_delta_u = horizontal_vec / image_width
pixel_delta_v = vertical_vec / image_height

In [ ]:
%%time

for row in range(image_height):
  for col in range(image_width):
    u = col + 0.5
    v = row + 0.5

    ray = Ray(origin=camera_origin,
              direction=(lower_left_corner + u*pixel_delta_u + v*pixel_delta_v) - camera_origin)

    image[row, col] = ray_color(ray)

구의 법선 벡터의 좌표 값이 색상으로 잘 나타나는지 확인합니다.

In [ ]:
plt.imshow(image, origin="lower")
plt.show()